**API Pulls from Codeforces.com**

In [2]:
import requests
import json
import pandas as pd

**User Rated List**

This code imports the requests and csv modules.
It uses requests.get(url) to send a GET request to the API endpoint and retrieves the JSON data.
The JSON response is parsed using response.json()["result"].
The necessary fields are defined in a list.
A CSV file is opened using csv.DictWriter and the specified field names.
The JSON data is iterated, and each item is written to a row in the CSV file using writer.writerow.
The CSV file is saved, and a message is printed to indicate whether the data has been successfully saved or if an error occurred.

In [ ]:
import requests
import csv

# Send a GET request to the API endpoint
response = requests.get("https://codeforces.com/api/user.ratedList")

if response.status_code != 200:
    print("Error: Failed to retrieve data from API")
else:
    # Parse the JSON response
    data = response.json()["result"]

    # Define the fields to extract from each item in the result
    fields = ["handle", "lastName", "firstName", "country", "city", "rating", "rank", "maxRating"]

    # Open a CSV file for writing
    filepath = "C:\\Users\\chfi21ab\\Desktop\\codeforces_data\\FinalTables\\AllUserInfo.csv"
    try:
        with open(filepath, "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fields)

            # Write the header row
            writer.writeheader()

            # Write each item in the result to a new row in the CSV file
            for item in data:
                row = {field: item.get(field, "") for field in fields}
                writer.writerow(row)

        print(f"Data successfully saved to {filepath}")
    except Exception as e:
        print(f"Error: Failed to save data to {filepath}: {e}")

**Contest Standings**

The code essentially retrieves contest standings data for a range of contest IDs from the Codeforces API and saves the data to a CSV file for further analysis or processing.

*Details:*

This code defines the API URL pattern for retrieving contest standings. It uses a placeholder {} to represent the contest ID, which will be replaced later.
Then it defines the range of contest IDs to retrieve, from min_contest_id to max_contest_id.
It code opens a CSV file for writing, specified by the filepath variable and creates a csv.writer object to write data to the CSV file. Next, the header row is written to the CSV file, containing the column names: "contestId", "rank", "handle", "rating", and "points".
The code loops through each contest ID in the range from min_contest_id to max_contest_id.
Inside the loop, it constructs the API URL for the current contest ID by replacing the placeholder in the contest_standings_url with the current contest ID.
A GET request is sent to the Codeforces API using requests.get() to retrieve the contest standings data for the current contest ID.
If the response status code is not 200 (indicating a successful request), it prints an error message indicating the failure to retrieve data for that particular contest ID. If the response status code is 200, the JSON response is parsed, and the standings_data containing the rows of standings is extracted from the response.
Each row of standings data is iterated, and specific fields such as "handle", "rating", "rank", and "points" are extracted.
The extracted data is written to a new row in the CSV file using writer.writerow().
After writing all the data for the current contest ID, it prints a success message indicating that the data for that contest ID has been retrieved and saved to the CSV file.
After the loop completes, it prints a final message indicating that all the data has been successfully saved to the specified filepath.

In [ ]:
import requests
import csv

# Define the API URL pattern with a placeholder for the contest ID
contest_standings_url = "https://codeforces.com/api/contest.standings?contestId={}&from=1&showUnofficial=true"

# Define the range of contest IDs to retrieve
min_contest_id = 1
max_contest_id = 1850

# Open a CSV file for writing
filepath = "C:\\Users\\chfi21ab\\Desktop\\codeforces_data\\FinalTables\\ContestStandingsNew.csv"
with open(filepath, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(["contestId", "rank", "handle", "rating", "points"])

    # Loop through all contest IDs in the range and retrieve the data from the API
    for contest_id in range(min_contest_id, max_contest_id + 1):
        # Construct the API URL with the current contest ID
        standings_url = contest_standings_url.format(contest_id)

        # Send a GET request to the standings API endpoint
        standings_response = requests.get(standings_url)

        if standings_response.status_code != 200:
            print(f"Error: Failed to retrieve standings data for contest ID {contest_id}")
        else:
            # Parse the JSON response
            standings_data = standings_response.json()["result"]["rows"]

            # Write each item in the result to a new row in the CSV file
            for row in standings_data:
                handle = row["party"]["members"][0]["handle"]
                rating = row.get("rating", "N/A")
                writer.writerow([contest_id, row["rank"], handle, rating, row["points"]])

            print(f"Data for contest ID {contest_id} successfully retrieved and saved to CSV file")

print(f"All data successfully saved to {filepath}")

**ContestHacks, ContestStandings, ContestList, and RatingChanges**

This code contains all functions to retrieve the API metodes ContestHacks, ContestStandings, ContestList and RatingChanges. Input is the respective ContestID.

In [6]:
import requests

import json

import pandas as pd

from typing import List



def get_contest_hacks(contest_id: int) -> pd.DataFrame:

    url = f"https://codeforces.com/api/contest.hacks?contestId={contest_id}"

    r = requests.get(url)

    json_data = r.json()

    if json_data['status'] != "OK":

        print(f"Error getting contest hacks for contest {contest_id}")

        return pd.DataFrame()

    else:

        df = pd.DataFrame(json_data['result'])

        df.to_csv(f"C:/Users/chfi21ab/Desktop/codeforces_data/ContestHacks/{contest_id}_hacks.csv", index=False)

        return df

def get_contest_standings(contest_id: int) -> pd.DataFrame:

    url = f"https://codeforces.com/api/contest.standings?contestId={contest_id}&from=1&showUnofficial=true"

    r = requests.get(url)

    json_data = r.json()

    if json_data['status'] != "OK":

        print(f"Error getting contest standings for contest {contest_id}")

        return pd.DataFrame()

    else:

        df = pd.DataFrame(json_data['result']['rows'])

        df.to_csv(f"C:/Users/chfi21ab/Desktop/codeforces_data/ContestStandings/{contest_id}_standings.csv", index=False)

        return df

def get_contest_list(gym: bool = False) -> List[dict]:

    url = "https://codeforces.com/api/contest.list"

    if gym:

        url += "?gym=true"

    r = requests.get(url)

    json_data = r.json()

    if json_data['status'] != "OK":

        print("Error getting contest list")

        return []

    else:

        df = pd.DataFrame(json_data['result'])

        df.to_csv("C:/Users/chfi21ab/Desktop/codeforces_data/ContestList.csv", index=False)

        return df

def get_rating_changes(contest_id: int) -> pd.DataFrame:

    url = f"https://codeforces.com/api/contest.ratingChanges?contestId={contest_id}"

    r = requests.get(url)

    json_data = r.json()

    if json_data['status'] != "OK":

        print(f"Error getting rating changes for contest {contest_id}")

        return pd.DataFrame()

    else:

        df = pd.DataFrame(json_data['result'])

        df.to_csv(f"C:/Users/chfi21ab/Desktop/codeforces_data/RatingChanges/{contest_id}_rating_changes.csv", index=False)
        return df

This code is used for the final execution of the functions defined above. At first only the functions ContestHacks, ContestStandings and RatingChanges are executed because they work with IDs and therefore we can use a loop. Finallz the Contest List Method is executed as it does not need an Id.

In [ ]:
import os

from time import sleep

# Set up folders to store data

os.makedirs('DATA/ContestHacks', exist_ok=True)

os.makedirs('DATA/ContestList', exist_ok=True)

os.makedirs('DATA/RatingChanges', exist_ok=True)

# Call API functions and save data
for contest_id in range(1, 1850):
    get_contest_hacks(contest_id)

    get_contest_standings(contest_id)

    get_rating_changes(contest_id)

    sleep(1) # Add a delay to avoid exceeding API limits

get_contest_list()